In [1]:
# Import the research library 
import logging
import benchmarklib
from benchmarklib import BenchmarkDatabase, CompileType, GroverRunner, GroverConfig, calculate_grover_iterations

benchmarklib.setup_logging(logging.INFO)

from clique import CliqueProblem, CliqueTrial, migrate_old_graph_to_new_system

db = BenchmarkDatabase("Clique.db", CliqueProblem, CliqueTrial)

benchmarklib.quantum_trials - INFO - Database initialized: Clique.db (CLIQUE)


In [2]:
# Load Qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Batch
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
API_INSTANCE = os.getenv("API_INSTANCE", None)
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN, instance=API_INSTANCE)
backend = service.backend(name="ibm_rensselaer")

/tmp/ipykernel_246707/2424275111.py:9: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN, instance=API_INSTANCE)


In [3]:
# Configure Grover Benchmark
config = GroverConfig(shots=10**4, optimization_level=3)
runner = GroverRunner(db_manager=db, service=service, backend=backend, config=config)

benchmarklib.grover - INFO - GroverRunner initialized for CLIQUE problems


In [6]:
nodes_range = range(7, 11)
edge_prob_range = range(1, 100, 10)
#edge_prob_range = range(70, 71)

In [7]:
# benchmark driver

for nodes in nodes_range:
    for prob in edge_prob_range:
        #with Batch(backend) as batch:
        size_filter = {'num_vertices': nodes, 'edge_probability': prob}
        for problem in db.find_problem_instances(
            size_filters=size_filter, 
            limit=20, 
            choose_untested=True,
            random_sample=True
        ):
            target_clique_size = max(nodes//2, 2)
            cliques_of_target_size = problem.clique_counts[target_clique_size]
            if cliques_of_target_size == 0:
                # clique of size target_clique_size DNE for this graph
                continue

            optimal_grover_iters = calculate_grover_iterations(cliques_of_target_size, 2**nodes)
            for compile_type in [CompileType.CLASSICAL_FUNCTION, CompileType.XAG]:
                for grover_iter in range(1, optimal_grover_iters):

                    # we have a trial for this problem and for each possible grover 
                    # iteration up to the optimal and for each compiler type

                    # saved to DB by GroverRunner
                    runner.run_grover_benchmark(
                        problem_instance=problem,
                        compile_type=compile_type,
                        grover_iterations=grover_iter,
                        # kwargs for oracle compilation
                        clique_size=target_clique_size
                    )
            

benchmarklib.grover - INFO - Problem Instance ID: 3339
benchmarklib.grover - INFO - Submitted job d1089rf3grvg008hms4g to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 1083
benchmarklib.grover - INFO - Problem Instance ID: 3339
benchmarklib.grover - INFO - Submitted job d1089s7mya70008e6f20 to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 1084
benchmarklib.grover - INFO - Problem Instance ID: 3339
benchmarklib.grover - INFO - Submitted job d1089szn2txg008jdmc0 to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 1085
benchmarklib.grover - INFO - Problem Instance ID: 3339
benchmarklib.grover - INFO - Submitted job d1089tz3grvg008hms8g to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 1086
benchmarklib.grover - INFO - Problem Instance ID: 3339
benchmarklib.grover - INFO - Submitted job d1089w73grvg008hmsb0 to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 1087
benchmarklib.grover - INFO - Problem Instance ID: 3339
benchmarklib.grover - INFO - S